# Chargement des données

In [1]:
# fichiers de https://parltrack.org/dumps

import json, collections

députés = json.load(open('ep_meps.json'))
députés_par_id = {député["UserID"]: député for député in députés}

tout_les_votes = json.load(open('ep_votes.json'))
print('données chargées')

données chargées


In [2]:
def nom(id):
    return députés_par_id[id]['Name']['full']


In [3]:
# deputés francais
# https://fr.wikipedia.org/wiki/Liste_des_d%C3%A9put%C3%A9s_europ%C3%A9ens_de_France_de_la_9e_l%C3%A9gislature
# extrait avec https://wikitable2csv.ggor.de/
import csv
from fuzzywuzzy import fuzz

députés_francais = list(csv.DictReader(open('liste_deputes_europeens.csv')))
IDs_à_garder = set()
for député in députés_francais:
    nom1 = député["Nom"]
    found_correspondance = False
    for autre_député in députés:
        nom2 = nom(autre_député["UserID"])
        if fuzz.token_set_ratio(nom2, nom1) > 90:
            # print('Correspondance:', nom2, '/', nom1)
            if found_correspondance:
                print('Double !')
            found_correspondance = autre_député
    if not found_correspondance:
        print('pas de correspondance:', nom1)
    député['données'] = found_correspondance
    IDs_à_garder.add(found_correspondance["UserID"])

députés = [député for député in députés if député["UserID"] in IDs_à_garder]
députés_par_id = {député["UserID"]: député for député in députés}
print(députés_par_id.keys())

/home/damien/.venv/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


dict_keys([96750, 22858, 96711, 190774, 96868, 24594, 96747, 189065, 5565, 30482, 72779, 96885, 124765, 96743, 96952, 96740, 24505, 113892, 124693, 124738, 124770, 124771, 124760, 197691, 197533, 131580, 197680, 197534, 197551, 197512, 197521, 197577, 182995, 197576, 197574, 197505, 197529, 197536, 197683, 197503, 197504, 197531, 197535, 197589, 197628, 197694, 94649, 197557, 197543, 135511, 197627, 197626, 197585, 197697, 197597, 197698, 197686, 197494, 197623, 197687, 197527, 197629, 197615, 197506, 197690, 197500, 197508, 197547, 97236, 197593, 197502, 197511, 197581, 197499, 204416, 204420, 204419, 204421, 204418])


In [9]:
votes_par_députés = {id_député: collections.defaultdict(list) for id_député in députés_par_id}

for vote in tout_les_votes:
    if 'votes' not in vote:
        continue
    date = vote['ts'].split('T')[0]
    if date < "2019-02-01":
        continue
    for position, position_votes in vote['votes'].items():
        for groupe, groupe_votes in position_votes['groups'].items():
            for député in groupe_votes:
                if 'mepid' not in député:
                    # print("Député sans ID:", député)
                    continue
                if député['mepid'] in votes_par_députés:
                    votes_par_députés[député['mepid']][date] = [1]

In [6]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [13]:
import datetime, statistics

# grouper par semaine

votes_par_mois = collections.defaultdict(dict)

for député, dates in votes_par_députés.items():
    for date, votes in dates.items():
        année, mois, jour = date.split('-')
        mois_complet = f"{année}-{mois}"
        if député not in votes_par_mois[mois_complet]:
            votes_par_mois[mois_complet][député] = [votes]
        votes_par_mois[mois_complet][député] += votes

def moyenne_votes(votes):
    return statistics.mean([len(votes.get(député, [])) for député in députés_par_id])

from bokeh.io import show
from bokeh.plotting import figure
        
tout_les_mois = sorted(votes_par_mois.keys())

def compte_votes(député):
    return sum([len(mois.get(député, [])) for mois in votes_par_mois.values()])

députés_les_moins_présents = sorted(votes_par_députés.keys(),
                                    key=compte_votes)


def affiche_député(député):
    votes = [len(votes_par_mois[mois].get(député, [])) for mois in tout_les_mois]
    votes_moyenne = [moyenne_votes(votes_par_mois[mois]) for mois in tout_les_mois]

    p = figure(x_range=tout_les_mois, plot_width=700, plot_height=400,
               title=f"Jours où {nom(député)} à voté ({compte_votes(député)})",
               )

    p.step(tout_les_mois, votes_moyenne, line_width=2, line_color="red", mode="before", alpha=0.8, legend_label='Moyenne de tous les députés français')
    p.vbar(x=tout_les_mois, top=votes, width=1, alpha=0.5)

    p.y_range.start = 0
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.xaxis.major_label_orientation = "vertical"
    show(p)


# Les 10 moins présents

In [14]:
for député in députés_les_moins_présents[:10]:
    affiche_député(député)
for député in députés_les_moins_présents[:10]:
    print(f"{nom(député)} avec {compte_votes(député)} jours")

Ilana CICUREL

Ilana CICUREL avec 21 jours
Claude GRUFFAT avec 21 jours
Sandro GOZI avec 21 jours
Jean-Lin LACAPELLE avec 21 jours
Nora MEBAREK avec 21 jours
Gilbert COLLARD avec 37 jours
Herve JUVIN avec 37 jours
Jean-Paul GARRAUD avec 40 jours
Maxette PIRBAKAS avec 41 jours
Agnès EVREN avec 44 jours


# Les 10 plus présents

In [15]:
for député in députés_les_moins_présents[-10:]:
    affiche_député(député)
for député in députés_les_moins_présents[-10:]:
    print(f"{nom(député)} avec {compte_votes(député)} jours")

Arnaud DANJEAN avec 66 jours
Gilles LEBRETON avec 66 jours
Karima DELLI avec 68 jours
Younous OMARJEE avec 68 jours
Pascal DURAND avec 69 jours
Dominique BILDE avec 69 jours
France JAMET avec 71 jours
Anne SANDER avec 73 jours
Dominique RIQUET avec 73 jours
Sylvie GUILLAUME avec 73 jours


In [6]:
# todo: trous dans les mois
# plot line not aligned, ligne date debut mandat